<a href="https://colab.research.google.com/github/jeremiahoclark/python-coding-patterns/blob/main/08_error_handling_logging_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Error Handling and Logging Patterns

This notebook covers essential patterns for robust error handling and effective logging in Python applications, including custom exceptions, graceful error recovery, and structured logging practices.

## 1. Exception Handling Strategies

Effective exception handling involves knowing when to catch, when to propagate, and how to use custom exceptions for clear error communication.

In [ ]:
# Custom Exception Hierarchy
import math
import logging
from typing import Any, Dict, List, Optional, Union
import traceback
from datetime import datetime
import json

# Base custom exception
class ApplicationError(Exception):
    """Base exception for application-specific errors"""
    def __init__(self, message: str, error_code: str = None, context: Dict[str, Any] = None):
        super().__init__(message)
        self.message = message
        self.error_code = error_code
        self.context = context or {}
        self.timestamp = datetime.now()

# Specific exception types
class ValidationError(ApplicationError):
    """Raised when input validation fails"""
    def __init__(self, message: str, field_errors: Dict[str, str] = None):
        super().__init__(message, "VALIDATION_ERROR", {"field_errors": field_errors or {}})
        self.field_errors = field_errors or {}

class BusinessLogicError(ApplicationError):
    """Raised when business rules are violated"""
    def __init__(self, message: str, rule: str = None):
        super().__init__(message, "BUSINESS_LOGIC_ERROR", {"rule": rule})
        self.rule = rule

class ResourceNotFoundError(ApplicationError):
    """Raised when a requested resource is not found"""
    def __init__(self, resource_type: str, identifier: Any):
        message = f"{resource_type} not found: {identifier}"
        super().__init__(message, "RESOURCE_NOT_FOUND", {
            "resource_type": resource_type,
            "identifier": identifier
        })
        self.resource_type = resource_type
        self.identifier = identifier

class ExternalServiceError(ApplicationError):
    """Raised when external service communication fails"""
    def __init__(self, service_name: str, operation: str, cause: Exception = None):
        message = f"External service error in {service_name}.{operation}"
        super().__init__(message, "EXTERNAL_SERVICE_ERROR", {
            "service_name": service_name,
            "operation": operation
        })
        self.service_name = service_name
        self.operation = operation
        self.cause = cause

class ConfigurationError(ApplicationError):
    """Raised when configuration is invalid or missing"""
    def __init__(self, config_key: str, expected: str = None):
        message = f"Invalid configuration: {config_key}"
        if expected:
            message += f" (expected: {expected})"
        super().__init__(message, "CONFIGURATION_ERROR", {
            "config_key": config_key,
            "expected": expected
        })

print("Custom exception hierarchy defined.")

In [ ]:
# Exception Handling Patterns and Strategies
import os
import time
import random
from functools import wraps
from contextlib import contextmanager

# Pattern 1: EAFP (Easier to Ask for Forgiveness than Permission)
class Calculator:
    """Calculator demonstrating EAFP and custom exceptions"""

    @staticmethod
    def safe_divide(numerator: float, denominator: float) -> float:
        """Divide two numbers with custom exception for zero division"""
        if denominator == 0:
            raise BusinessLogicError(
                "Division by zero is not allowed",
                rule="non_zero_denominator"
            )
        return numerator / denominator

    @staticmethod
    def safe_sqrt(number: float) -> float:
        """Calculate square root with validation"""
        if number < 0:
            raise ValidationError(
                "Cannot calculate square root of negative number",
                field_errors={"number": "Must be non-negative"}
            )
        return math.sqrt(number)

    @staticmethod
    def safe_log(number: float, base: float = math.e) -> float:
        """Calculate logarithm with validation"""
        errors = {}
        if number <= 0:
            errors["number"] = "Must be positive"
        if base <= 0 or base == 1:
            errors["base"] = "Must be positive and not equal to 1"

        if errors:
            raise ValidationError("Invalid logarithm parameters", field_errors=errors)

        return math.log(number, base)

# Pattern 2: Retry with exponential backoff
def retry_with_backoff(max_attempts: int = 3, base_delay: float = 1.0,
                      backoff_factor: float = 2.0, exceptions: tuple = None):
    """Decorator for retrying operations with exponential backoff"""
    if exceptions is None:
        exceptions = (ExternalServiceError,)

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None

            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    last_exception = e
                    if attempt == max_attempts - 1:
                        # Last attempt, re-raise
                        raise

                    delay = base_delay * (backoff_factor ** attempt)
                    print(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay:.1f}s...")
                    time.sleep(delay)

            # This should never be reached, but just in case
            raise last_exception

        return wrapper
    return decorator

# Pattern 3: Context manager for resource cleanup
@contextmanager
def temporary_config(config_changes: Dict[str, Any]):
    """Context manager for temporary configuration changes"""
    original_values = {}

    # Save original values and apply changes
    for key, new_value in config_changes.items():
        original_values[key] = os.environ.get(key)
        os.environ[key] = str(new_value)

    try:
        yield
    except Exception as e:
        print(f"Exception occurred during temporary config: {e}")
        raise
    finally:
        # Restore original values
        for key, original_value in original_values.items():
            if original_value is None:
                os.environ.pop(key, None)
            else:
                os.environ[key] = original_value

# Pattern 4: Exception chaining
class DataProcessor:
    """Demonstrates exception chaining and context preservation"""

    def parse_number(self, value: str) -> float:
        """Parse string to number with context preservation"""
        try:
            return float(value)
        except ValueError as e:
            raise ValidationError(
                f"Invalid number format: '{value}'",
                field_errors={"value": "Must be a valid number"}
            ) from e

    def load_config_file(self, filename: str) -> Dict[str, Any]:
        """Load configuration from file with proper error handling"""
        try:
            with open(filename, 'r') as f:
                data = json.load(f)
                return data
        except FileNotFoundError as e:
            raise ConfigurationError(f"config_file:{filename}", "existing file") from e
        except json.JSONDecodeError as e:
            raise ConfigurationError(
                f"config_file:{filename}",
                "valid JSON format"
            ) from e
        except PermissionError as e:
            raise ConfigurationError(
                f"config_file_permissions:{filename}",
                "read permissions"
            ) from e

print("Exception handling patterns defined.")

In [ ]:
# Demonstrate exception handling patterns
print("=== Exception Handling Patterns Demo ===")

calculator = Calculator()
processor = DataProcessor()

# Test 1: Basic custom exceptions
print("\n1. Testing custom exceptions:")
test_cases = [
    ("safe_divide", [10, 2]),
    ("safe_divide", [10, 0]),  # Should raise BusinessLogicError
    ("safe_sqrt", [16]),
    ("safe_sqrt", [-4]),  # Should raise ValidationError
    ("safe_log", [100, 10]),
    ("safe_log", [-5, 10]),  # Should raise ValidationError
]

for method_name, args in test_cases:
    try:
        method = getattr(calculator, method_name)
        result = method(*args)
        print(f"  {method_name}({', '.join(map(str, args))}) = {result}")
    except ApplicationError as e:
        print(f"  {method_name}({', '.join(map(str, args))}) -> {type(e).__name__}: {e.message}")
        if hasattr(e, 'field_errors') and e.field_errors:
            print(f"    Field errors: {e.field_errors}")

# Test 2: Exception chaining
print("\n2. Testing exception chaining:")
try:
    number = processor.parse_number("not_a_number")
except ValidationError as e:
    print(f"Caught: {type(e).__name__}: {e.message}")
    print(f"Original cause: {type(e.__cause__).__name__}: {e.__cause__}")

# Test 3: Context manager for cleanup
print("\n3. Testing context manager:")
print(f"Original PATH length: {len(os.environ.get('PATH', ''))}")

try:
    with temporary_config({"TEST_VAR": "temporary_value", "PATH": "/tmp"}):
        print(f"Temporary PATH: {os.environ.get('PATH')}")
        print(f"TEST_VAR: {os.environ.get('TEST_VAR')}")
        # Simulate an error
        raise ValueError("Simulated error")
except ValueError as e:
    print(f"Caught error: {e}")

print(f"Restored PATH length: {len(os.environ.get('PATH', ''))}")
print(f"TEST_VAR after cleanup: {os.environ.get('TEST_VAR', 'Not set')}")

In [ ]:
# Demonstrate retry pattern
print("\n4. Testing retry pattern:")

class UnreliableService:
    """Simulates an unreliable external service"""

    def __init__(self, failure_rate: float = 0.7):
        self.failure_rate = failure_rate
        self.attempt_count = 0

    @retry_with_backoff(max_attempts=3, base_delay=0.1, exceptions=(ExternalServiceError,))
    def unreliable_operation(self, data: str) -> str:
        """Operation that fails randomly"""
        self.attempt_count += 1
        print(f"    Attempting operation (attempt {self.attempt_count})...")

        if random.random() < self.failure_rate:
            raise ExternalServiceError("payment_service", "process_payment",
                                     cause=Exception("Network timeout"))

        return f"Success: Processed '{data}'"

    def reset(self):
        self.attempt_count = 0

# Test retry with different failure rates
service = UnreliableService(failure_rate=0.6)  # 60% failure rate

for i in range(3):
    print(f"\nTest {i+1}:")
    service.reset()
    try:
        result = service.unreliable_operation(f"test_data_{i}")
        print(f"  Result: {result}")
    except ExternalServiceError as e:
        print(f"  Final failure: {e.message}")
    print(f"  Total attempts made: {service.attempt_count}")

In [ ]:
# Advanced error handling patterns
from contextlib import suppress
from typing import Type, Callable, Any

# Pattern 5: Graceful degradation
class FeatureManager:
    """Manages optional features with graceful degradation"""

    def __init__(self):
        self.features = {
            'analytics': True,
            'caching': True,
            'notifications': True
        }

    def safe_feature_call(self, feature_name: str, operation: Callable,
                         fallback: Any = None, critical: bool = False):
        """Execute feature operation with graceful degradation"""
        if not self.features.get(feature_name, False):
            print(f"Feature '{feature_name}' is disabled, using fallback")
            return fallback

        try:
            return operation()
        except Exception as e:
            if critical:
                print(f"Critical feature '{feature_name}' failed: {e}")
                raise
            else:
                print(f"Optional feature '{feature_name}' failed: {e}, using fallback")
                self.features[feature_name] = False  # Disable failing feature
                return fallback

    def send_analytics(self, event: str):
        """Send analytics event (optional feature)"""
        def analytics_operation():
            # Simulate analytics service that might fail
            if random.random() < 0.3:  # 30% failure rate
                raise ExternalServiceError("analytics_service", "send_event")
            return f"Analytics sent: {event}"

        return self.safe_feature_call(
            'analytics',
            analytics_operation,
            fallback="Analytics disabled",
            critical=False
        )

    def process_payment(self, amount: float):
        """Process payment (critical feature)"""
        def payment_operation():
            if amount <= 0:
                raise ValidationError("Amount must be positive")
            return f"Payment of ${amount} processed"

        return self.safe_feature_call(
            'payment',
            payment_operation,
            fallback=None,
            critical=True
        )

# Pattern 6: Error suppression for cleanup
class ResourceManager:
    """Demonstrates safe resource cleanup"""

    def __init__(self):
        self.resources = []

    def acquire_resource(self, name: str):
        """Acquire a resource"""
        print(f"Acquiring resource: {name}")
        self.resources.append(name)
        return name

    def release_resource(self, name: str):
        """Release a resource (might fail)"""
        if "bad" in name:
            raise Exception(f"Failed to release {name}")
        print(f"Released resource: {name}")
        self.resources.remove(name)

    def cleanup_all_resources(self):
        """Clean up all resources, ignoring individual failures"""
        errors = []

        for resource in self.resources.copy():
            try:
                self.release_resource(resource)
            except Exception as e:
                errors.append(f"Failed to release {resource}: {e}")
                # Still remove from our list even if release failed
                with suppress(ValueError):
                    self.resources.remove(resource)

        if errors:
            print(f"Cleanup completed with {len(errors)} errors:")
            for error in errors:
                print(f"  - {error}")
        else:
            print("All resources cleaned up successfully")

        return len(errors) == 0

print("Advanced error handling patterns defined.")

In [ ]:
# Test advanced error handling patterns
print("=== Advanced Error Handling Demo ===")

# Test graceful degradation
print("\n1. Testing graceful degradation:")
feature_manager = FeatureManager()

# Send multiple analytics events (some may fail)
events = ["user_login", "page_view", "button_click", "form_submit", "user_logout"]
for event in events:
    result = feature_manager.send_analytics(event)
    print(f"  {result}")

print(f"\nFeature status after tests: {feature_manager.features}")

# Test critical vs non-critical errors
print("\n2. Testing critical vs non-critical errors:")
try:
    result = feature_manager.process_payment(100.0)
    print(f"  {result}")
except Exception as e:
    print(f"  Critical error caught: {e}")

try:
    result = feature_manager.process_payment(-50.0)  # Invalid amount
    print(f"  {result}")
except ValidationError as e:
    print(f"  Validation error caught: {e.message}")

# Test resource cleanup with error suppression
print("\n3. Testing resource cleanup:")
resource_manager = ResourceManager()

# Acquire some resources
resources = ["database_connection", "file_handle", "bad_network_connection", "cache_connection"]
for resource in resources:
    resource_manager.acquire_resource(resource)

print(f"\nAcquired resources: {resource_manager.resources}")

# Clean up all resources
print("\nCleaning up resources:")
success = resource_manager.cleanup_all_resources()
print(f"Cleanup successful: {success}")
print(f"Remaining resources: {resource_manager.resources}")

## 2. Logging Best Practices

Effective logging provides visibility into application behavior, aids debugging, and supports monitoring and alerting.

In [ ]:
# Comprehensive logging setup and patterns
import logging
import logging.handlers
import sys
import json
import uuid
from datetime import datetime
from typing import Any, Dict, Optional
import threading
import functools

# Custom JSON formatter for structured logging
class JSONFormatter(logging.Formatter):
    """Custom formatter to output logs in JSON format"""

    def format(self, record):
        log_entry = {
            'timestamp': datetime.fromtimestamp(record.created).isoformat(),
            'level': record.levelname,
            'logger': record.name,
            'message': record.getMessage(),
            'module': record.module,
            'function': record.funcName,
            'line': record.lineno,
        }

        # Add process and thread info
        log_entry['process_id'] = record.process
        log_entry['thread_id'] = record.thread

        # Add exception info if present
        if record.exc_info:
            log_entry['exception'] = self.formatException(record.exc_info)

        # Add extra fields
        for key, value in record.__dict__.items():
            if key not in ['name', 'msg', 'args', 'levelname', 'levelno', 'pathname',
                          'filename', 'module', 'exc_info', 'exc_text', 'stack_info',
                          'lineno', 'funcName', 'created', 'msecs', 'relativeCreated',
                          'thread', 'threadName', 'processName', 'process', 'getMessage']:
                log_entry[key] = value

        return json.dumps(log_entry, default=str)

# Context manager for request/operation tracking
class LoggingContext:
    """Context manager for adding context to all logs within a scope"""

    _context = threading.local()

    def __init__(self, **context):
        self.context = context
        self.previous_context = None

    def __enter__(self):
        self.previous_context = getattr(self._context, 'data', {})
        new_context = self.previous_context.copy()
        new_context.update(self.context)
        self._context.data = new_context
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self._context.data = self.previous_context

    @classmethod
    def get_context(cls) -> Dict[str, Any]:
        return getattr(cls._context, 'data', {})

# Custom log filter to add context
class ContextFilter(logging.Filter):
    """Filter that adds context information to log records"""

    def filter(self, record):
        context = LoggingContext.get_context()
        for key, value in context.items():
            setattr(record, key, value)
        return True

# Logging configuration manager
class LoggingManager:
    """Centralized logging configuration and management"""

    def __init__(self):
        self.loggers = {}

    def setup_logging(self,
                     level: str = 'INFO',
                     console_output: bool = True,
                     file_output: Optional[str] = None,
                     json_format: bool = False,
                     max_file_size: int = 10 * 1024 * 1024,  # 10MB
                     backup_count: int = 5):
        """Set up logging configuration"""

        # Clear existing handlers
        root_logger = logging.getLogger()
        root_logger.handlers.clear()

        # Set level
        root_logger.setLevel(getattr(logging, level.upper()))

        # Create formatter
        if json_format:
            formatter = JSONFormatter()
        else:
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )

        # Console handler
        if console_output:
            console_handler = logging.StreamHandler(sys.stdout)
            console_handler.setFormatter(formatter)
            console_handler.addFilter(ContextFilter())
            root_logger.addHandler(console_handler)

        # File handler with rotation
        if file_output:
            file_handler = logging.handlers.RotatingFileHandler(
                file_output,
                maxBytes=max_file_size,
                backupCount=backup_count
            )
            file_handler.setFormatter(formatter)
            file_handler.addFilter(ContextFilter())
            root_logger.addHandler(file_handler)

    def get_logger(self, name: str) -> logging.Logger:
        """Get a logger instance"""
        if name not in self.loggers:
            self.loggers[name] = logging.getLogger(name)
        return self.loggers[name]

    def set_logger_level(self, logger_name: str, level: str):
        """Set specific logger level"""
        logger = self.get_logger(logger_name)
        logger.setLevel(getattr(logging, level.upper()))

# Initialize logging manager
logging_manager = LoggingManager()
logging_manager.setup_logging(level='DEBUG', json_format=False)

print("Logging framework configured.")

In [ ]:
# Logging patterns and decorators
import time
import inspect
from functools import wraps

# Decorator for automatic logging of function calls
def log_function_calls(level: str = 'DEBUG', include_args: bool = True, include_result: bool = True):
    """Decorator to automatically log function calls"""
    def decorator(func):
        logger = logging_manager.get_logger(func.__module__)
        log_level = getattr(logging, level.upper())

        @wraps(func)
        def wrapper(*args, **kwargs):
            func_name = func.__name__

            # Log function entry
            if include_args:
                # Get argument names
                sig = inspect.signature(func)
                bound_args = sig.bind(*args, **kwargs)
                bound_args.apply_defaults()
                args_str = ', '.join(f'{k}={v}' for k, v in bound_args.arguments.items())
                logger.log(log_level, f"Calling {func_name}({args_str})")
            else:
                logger.log(log_level, f"Calling {func_name}")

            start_time = time.time()

            try:
                result = func(*args, **kwargs)
                execution_time = time.time() - start_time

                # Log successful completion
                if include_result:
                    logger.log(log_level,
                             f"{func_name} completed in {execution_time:.3f}s, result: {result}")
                else:
                    logger.log(log_level,
                             f"{func_name} completed in {execution_time:.3f}s")

                return result

            except Exception as e:
                execution_time = time.time() - start_time
                logger.error(f"{func_name} failed after {execution_time:.3f}s: {e}", exc_info=True)
                raise

        return wrapper
    return decorator

# Performance logging decorator
def log_performance(threshold_ms: float = 100.0):
    """Log performance metrics for slow operations"""
    def decorator(func):
        logger = logging_manager.get_logger(f"{func.__module__}.performance")

        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            execution_time = (time.time() - start_time) * 1000  # Convert to ms

            if execution_time > threshold_ms:
                logger.warning(
                    f"Slow operation detected: {func.__name__} took {execution_time:.2f}ms",
                    extra={
                        'function': func.__name__,
                        'execution_time_ms': execution_time,
                        'threshold_ms': threshold_ms
                    }
                )
            else:
                logger.debug(
                    f"Performance: {func.__name__} took {execution_time:.2f}ms",
                    extra={
                        'function': func.__name__,
                        'execution_time_ms': execution_time
                    }
                )

            return result

        return wrapper
    return decorator

# Business logic with logging
class UserService:
    """User service with comprehensive logging"""

    def __init__(self):
        self.logger = logging_manager.get_logger(f"{__name__}.UserService")
        self.users = {
            1: {'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'active': True},
            2: {'id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'active': True},
            3: {'id': 3, 'name': 'Charlie', 'email': 'charlie@example.com', 'active': False},
        }

    @log_function_calls(level='INFO', include_args=True)
    def get_user(self, user_id: int) -> Dict[str, Any]:
        """Get user by ID with logging"""
        self.logger.debug(f"Looking up user with ID: {user_id}")

        if user_id not in self.users:
            self.logger.warning(f"User not found: {user_id}")
            raise ResourceNotFoundError("User", user_id)

        user = self.users[user_id]
        self.logger.info(f"Successfully retrieved user: {user['name']}")
        return user

    @log_function_calls(level='INFO')
    @log_performance(threshold_ms=50)
    def search_users(self, query: str) -> List[Dict[str, Any]]:
        """Search users by name or email"""
        self.logger.info(f"Searching users with query: '{query}'")

        # Simulate some processing time
        time.sleep(0.1)  # 100ms delay to trigger performance warning

        query_lower = query.lower()
        results = []

        for user in self.users.values():
            if (query_lower in user['name'].lower() or
                query_lower in user['email'].lower()):
                results.append(user)

        self.logger.info(f"Search returned {len(results)} results")
        return results

    @log_function_calls(level='INFO')
    def update_user_status(self, user_id: int, active: bool) -> bool:
        """Update user active status"""
        try:
            user = self.get_user(user_id)  # This will log the lookup
            old_status = user['active']
            user['active'] = active

            self.logger.info(
                f"Updated user {user_id} status from {old_status} to {active}",
                extra={
                    'user_id': user_id,
                    'old_status': old_status,
                    'new_status': active,
                    'operation': 'status_update'
                }
            )
            return True

        except ResourceNotFoundError as e:
            self.logger.error(f"Failed to update user status: {e.message}")
            raise

    def process_bulk_operation(self, user_ids: List[int], operation: str):
        """Process bulk operation with detailed logging"""
        operation_id = str(uuid.uuid4())[:8]

        with LoggingContext(operation_id=operation_id, operation_type=operation):
            self.logger.info(f"Starting bulk operation: {operation} on {len(user_ids)} users")

            successful = 0
            failed = 0

            for user_id in user_ids:
                with LoggingContext(current_user_id=user_id):
                    try:
                        if operation == 'deactivate':
                            self.update_user_status(user_id, False)
                        elif operation == 'activate':
                            self.update_user_status(user_id, True)
                        else:
                            raise ValidationError(f"Unknown operation: {operation}")

                        successful += 1

                    except Exception as e:
                        failed += 1
                        self.logger.error(f"Failed to process user {user_id}: {e}")

            self.logger.info(
                f"Bulk operation completed: {successful} successful, {failed} failed",
                extra={
                    'successful_count': successful,
                    'failed_count': failed,
                    'total_count': len(user_ids)
                }
            )

print("Logging patterns and decorators defined.")

In [ ]:
# Demonstrate logging patterns
print("=== Logging Patterns Demo ===")

user_service = UserService()

print("\n1. Basic operations with automatic logging:")

# Test successful operations
try:
    user = user_service.get_user(1)
    print(f"Retrieved user: {user['name']}")
except Exception as e:
    print(f"Error: {e}")

# Test error case
try:
    user = user_service.get_user(999)
except ResourceNotFoundError as e:
    print(f"Expected error: {e.message}")

print("\n2. Performance logging:")
results = user_service.search_users("alice")
print(f"Search results: {len(results)} users found")

print("\n3. Context-aware logging (bulk operations):")
user_service.process_bulk_operation([1, 2, 999, 3], 'deactivate')

print("\n4. Different log levels:")
logger = logging_manager.get_logger("demo")

logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning message")
logger.error("This is an error message")

# Log with extra context
logger.info(
    "User action performed",
    extra={
        'user_id': 123,
        'action': 'profile_update',
        'ip_address': '192.168.1.100',
        'user_agent': 'Mozilla/5.0...'
    }
)

print("\n5. Exception logging:")
try:
    # Cause an exception to demonstrate exception logging
    result = 1 / 0
except ZeroDivisionError as e:
    logger.exception("Mathematical error occurred")
    print("Exception logged with full traceback")

In [ ]:
# Demonstrate structured logging with JSON format
print("\n=== Structured Logging Demo ===")

# Reconfigure logging for JSON output
logging_manager.setup_logging(level='INFO', json_format=True)

# Create a new logger for structured logging demo
structured_logger = logging_manager.get_logger("structured_demo")

print("\nJSON formatted logs:")

# Log with context
with LoggingContext(request_id="req_123", user_id="user_456"):
    structured_logger.info("Processing user request")

    with LoggingContext(step="validation"):
        structured_logger.debug("Validating input parameters")
        structured_logger.info("Validation completed successfully")

    with LoggingContext(step="business_logic"):
        structured_logger.info("Executing business logic")

        # Simulate an error
        try:
            raise BusinessLogicError("Insufficient balance", rule="minimum_balance")
        except BusinessLogicError as e:
            structured_logger.error(
                "Business rule violation",
                extra={
                    'error_code': e.error_code,
                    'rule': e.rule,
                    'balance': 50.0,
                    'required': 100.0
                }
            )

# Log metrics
structured_logger.info(
    "Request completed",
    extra={
        'duration_ms': 234.5,
        'status_code': 400,
        'bytes_sent': 1024,
        'cache_hit': False
    }
)

# Reset to regular logging for remaining demos
logging_manager.setup_logging(level='INFO', json_format=False)
print("\n(Logging format reset to regular text)")

In [ ]:
# Advanced logging patterns
print("\n=== Advanced Logging Patterns ===")

# Rate-limited logging to prevent log spam
class RateLimitedLogger:
    """Logger that rate-limits repeated messages"""

    def __init__(self, logger: logging.Logger, max_messages: int = 5, window_seconds: int = 60):
        self.logger = logger
        self.max_messages = max_messages
        self.window_seconds = window_seconds
        self.message_counts = {}
        self.last_reset = time.time()

    def _reset_if_needed(self):
        current_time = time.time()
        if current_time - self.last_reset > self.window_seconds:
            self.message_counts.clear()
            self.last_reset = current_time

    def log(self, level: int, message: str, *args, **kwargs):
        """Log message with rate limiting"""
        self._reset_if_needed()

        # Use message template as key for rate limiting
        message_key = message % args if args else message

        count = self.message_counts.get(message_key, 0)

        if count < self.max_messages:
            self.logger.log(level, message, *args, **kwargs)
            self.message_counts[message_key] = count + 1
        elif count == self.max_messages:
            self.logger.log(
                level,
                f"Rate limit reached for message: '{message_key}'. Suppressing further occurrences.",
                **kwargs
            )
            self.message_counts[message_key] = count + 1

    def info(self, message: str, *args, **kwargs):
        self.log(logging.INFO, message, *args, **kwargs)

    def warning(self, message: str, *args, **kwargs):
        self.log(logging.WARNING, message, *args, **kwargs)

    def error(self, message: str, *args, **kwargs):
        self.log(logging.ERROR, message, *args, **kwargs)

# Sampling logger for high-frequency events
class SamplingLogger:
    """Logger that samples high-frequency events"""

    def __init__(self, logger: logging.Logger, sample_rate: float = 0.1):
        self.logger = logger
        self.sample_rate = sample_rate
        self.total_events = 0
        self.logged_events = 0

    def log_event(self, level: int, message: str, *args, **kwargs):
        """Log event with sampling"""
        self.total_events += 1

        if random.random() < self.sample_rate:
            # Add sampling info to the log
            extra = kwargs.get('extra', {})
            extra.update({
                'sampled': True,
                'sample_rate': self.sample_rate,
                'total_events': self.total_events
            })
            kwargs['extra'] = extra

            self.logger.log(level, message, *args, **kwargs)
            self.logged_events += 1

    def get_stats(self):
        return {
            'total_events': self.total_events,
            'logged_events': self.logged_events,
            'sample_rate': self.sample_rate
        }

# Test rate-limited logging
print("\n1. Rate-limited logging demo:")
base_logger = logging_manager.get_logger("rate_limited_demo")
rate_limited_logger = RateLimitedLogger(base_logger, max_messages=3, window_seconds=60)

# Simulate repeated error messages
for i in range(8):
    rate_limited_logger.error("Database connection failed: attempt %d", i)

# Test sampling logger
print("\n2. Sampling logger demo:")
sampling_logger = SamplingLogger(
    logging_manager.get_logger("sampling_demo"),
    sample_rate=0.3  # Log 30% of events
)

# Simulate high-frequency events
for i in range(20):
    sampling_logger.log_event(
        logging.INFO,
        "High frequency event: %d",
        i,
        extra={'event_type': 'page_view', 'user_id': f'user_{i % 5}'}
    )

stats = sampling_logger.get_stats()
print(f"\nSampling stats: {stats}")

# Security-aware logging
print("\n3. Security-aware logging:")
security_logger = logging_manager.get_logger("security")

def safe_log_user_data(user_data: Dict[str, Any]):
    """Log user data while masking sensitive information"""
    # Fields to mask completely
    sensitive_fields = {'password', 'ssn', 'credit_card'}
    # Fields to partially mask
    partial_mask_fields = {'email', 'phone'}

    safe_data = {}

    for key, value in user_data.items():
        if key.lower() in sensitive_fields:
            safe_data[key] = '[REDACTED]'
        elif key.lower() in partial_mask_fields and isinstance(value, str):
            if '@' in value:  # Email
                parts = value.split('@')
                safe_data[key] = f"{parts[0][:2]}***@{parts[1]}"
            else:  # Phone or other
                safe_data[key] = f"{value[:3]}***{value[-2:]}" if len(value) > 5 else '[MASKED]'
        else:
            safe_data[key] = value

    security_logger.info("User data logged", extra={'user_data': safe_data})

# Test security logging
test_user_data = {
    'id': 123,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '555-123-4567',
    'password': 'secret123',
    'ssn': '123-45-6789'
}

safe_log_user_data(test_user_data)

print("\nAdvanced logging patterns demonstrated.")

## Key Takeaways

1. **Custom Exceptions**: Create specific exception types for clear error communication and handling
2. **Exception Chaining**: Preserve context when re-raising exceptions using `raise X from Y`
3. **Graceful Degradation**: Handle non-critical failures gracefully while maintaining core functionality
4. **Structured Logging**: Use consistent formats and include contextual information for better debugging
5. **Performance Logging**: Monitor slow operations and set appropriate thresholds
6. **Security Awareness**: Never log sensitive information; mask or redact as needed

## Best Practices

### Exception Handling
- Use specific exception types rather than catching broad `Exception`
- Implement retry logic for transient failures
- Always clean up resources using context managers or try/finally
- Log exceptions with sufficient context for debugging
- Consider the call stack when deciding where to handle exceptions

### Logging
- Configure logging early in application startup
- Use appropriate log levels (DEBUG, INFO, WARNING, ERROR, CRITICAL)
- Include contextual information in log messages
- Use structured logging for better searchability
- Implement log rotation to prevent disk space issues
- Be mindful of performance impact in high-frequency code paths

## Exercises

1. Create a custom exception hierarchy for a banking application
2. Implement a retry decorator with exponential backoff for API calls
3. Build a logging configuration that outputs to both console and rotating files
4. Create a context manager for database transactions with proper error handling
5. Implement a sampling logger for high-frequency events
6. Design a security-aware logger that automatically masks sensitive data